# Writer Monad

## Outline

* Incentive for the Writer monad

* The Writer monad
  - Definition of the Writer monad
  
  - Helper functions
  
  - MonadWriter type class
  
  - Simple example

* Writer monad examples

In this lesson, we will learn about the Writer monad type and how you can use it. First we present the reason why introducing the Writer monad. Then we show the Writer monad definition and helper functions. We also look at the `MonadWriter` type class and in the end show some code examples that use the Writer monad.

## Incentive for Writer monad

In the previous lecture we learned that with the Reader monad we can more easily pass data around by accessing the environment variable of the Reader monad. 

But what if instead of reading that data we want to append data to our environment variable?

Let's look at an example where we want to test 3 different list sorting algorithms. We want to know what are the counts of recursive calls for each algorithm.

We choose to implement the selection, quick and merge sort algorithems. We test them on three different list that have different patterns of unsorted elements.

In [ ]:
type List = [Int]
type OrderedList = [[Int]]

selectionSort :: List -> OrderedList 
selectionSort []   = [] 
selectionSort list = replicate count minElem : selectionSort rest
  where minElem = minimum list
        count = length $ filter (== minElem) list
        rest = filter (/= minElem) list

quicksort :: List -> OrderedList 
quicksort []     = [] 
quicksort (x:xs) = quicksort lesser ++ 
                   [[x]] ++ quicksort greater
  where 
    lesser  = filter (< x) xs 
    greater = filter (>= x) xs 

mergeSort :: List -> OrderedList
mergeSort [] = []
mergeSort [a] = [[a]]
mergeSort xs = merge (mergeSort (firstHalf xs)) (mergeSort (secondHalf xs))
  where merge xs [] = xs
        merge [] ys = ys
        merge (x:xs) (y:ys) | x <= y    = x:merge xs (y:ys)
                            | otherwise = y:merge (x:xs) ys
        firstHalf  xs = let { n = length xs } in take (div n 2) xs
        secondHalf xs = let { n = length xs } in drop (div n 2) xs

testSorts :: List -> IO ()
testSorts list = do
    let sorted1 = selectionSort list
        sorted2 = quicksort list
        sorted3 = mergeSort list
    
    putStrLn $ "Counts of recursive calls for the list " ++ show list
    putStrLn $ "Selection sort: " ++ show (length sorted1)
    putStrLn $ "Quick sort: " ++ show (length sorted2)
    putStrLn $ "Merge sort: " ++ show (length sorted3)
    putStrLn ""

main :: IO ()
main = do
    let list1 = [7, 6, 5, 4, 3, 2, 1]
        list2 = [3, 2, 1, 4, 1, 2, 3]
        list3 = [6, 3, 4, 2, 1, 3, 5]
        
    testSorts list1
    testSorts list2
    testSorts list3

main

We could also skip computing the sorted lists and introduce the following type signature for all of the sort functions:
```haskell
type Count = Int
sortFunction :: (Count, List) -> Count
```

Then we would need to pass on only the rest of the list and update the `Count` variable accordingly.

But when looking at the original sort functions it is less work to modify them as we did in our example.

In any case an obstacle we face is that we can not directly re-use the existing online code implementations of the sort algorithms. 

We need to figure out a way how to adapt them such that we can get the counts of recursive calls in the end. 

The second obstacle is that for a another developer reading our code it makes it harder to understand what it does.

If we would have a simple way to increase an `Int` environment variable for 1 after each recursive call, it would be easier.

For this reason the **Writer monad** was created that allows you to easily pass data between different Writer monads and write to it. 

## The Writer monad

### Definition of the Writer monad

The Haskell definition of the `Writer` type is defined in terms of the `WriterT` monad transformer:
```haskell
type Writer w = WriterT w Identity
```

It is contained in the **Control.Monad.Trans.Writer** module that comes with the **transformers** package. 

As said in the previous lesson we will learn about Monad transformers in lesson 25.

You can also define the Writer monad without using a monad transformer that works equally as the one above. 

Also in this lesson will chose this aproach. Let's first look at the definition of the **Writer** type:
```haskell
newtype Writer w a = Writer {runWriter :: (a,w)}
```

We see that the `Writer` data constructor holds a variable that can be accessed with the name `runWriter`.

The variable is a tuple of type `(a,w)`.

We can now create a Monad instance for `Writer w` and not just `Writer`.

This means the type of our writing variable will remain the same as we compose our function with `(>>=)`.
```haskell
(>>=) :: Writer w a        ->
         (a -> Writer w b) ->
         Writer w b
```

The monad instance for `Writer w` would be:
```haskell
instance (Monoid w) => Monad (Writer w) where
    return a             = Writer (a, mempty)
    (Writer (a,w)) >>= f = let (a',w') = runWriter $ f a 
                           in Writer (a', w `mappend` w')
```

We also need to define the Functor and Applicative instances.
```haskell
instance Functor (Writer w) where
fmap f (Writer (a,w)) = Writer (f a, w)

instance (Monoid w) => Applicative (Writer w) where
pure a = Writer (a, mempty)
(Writer (f,w)) <*> (Writer (a,w')) =  
    let b = f a
    in Writer (b, w `mappend` w')
```

### Helper functions

The **Control.Monad.Trans.Writer** module also defines some helper functions for the Writer moand. 

You have the option to import this module as a lazy or a strict module by appending **.Lazy** or **.Strict** to the module name.

The `tell` function appends data to the writer variable. It is used inside of the Writer monad.
```haskell
tell :: Monoid w => w -> Writer w ()
tell msg = Writer ((),msg)
```

The `writer` function does the oposite as the `runWriter` function. It takes a tuple and creates a Writer monad.
```haskell
writer :: Monoid w => (a,w) -> Writer w a
writer ~(a, w) = do
    tell w
    return a
```

The `listen` function retrieves the writer variable and the variable of type `a` from the Writer monad.
```haskell
listen :: Monoid w => Writer w a -> Writer w (a, w)
listen (Writer (a,w)) = Writer ((a,w),w)
```

It can be used inside or outside of a Writer Monad. To get both variable you also need to use the `<-` oprator that takes variables out of the monadic context.
```haskell
(a,w) <- listen writerMonad
```

The `pass` function takes in a Writer monad that holds a function which then gets applied to the writer variable and the updated Writer monad is returned. 
```haskell
pass :: Monoid w => Writer (a, w -> w) -> Writer w a
pass (Writer ((a,f),w)) = Writer (a,f w)
```

It can be used inside our outside or a Writer monad. 

The `listens` function takes in a function and a writer monad. Then it applies the function to the writer variable and returns the updated writer monad.
```haskell
listens :: Monoid w => (w -> b) -> Writer w a -> Writer w (a, b)
listens f writerMonad = do
    ~(a, w) <- listen writerMonad
    return (a, f w)
```

The tilde `~` sign in this expression means ... TODO

The `censor` function takes in a function and a writer monad and applies this function to the writer variable to update the writer monad.
```haskell
censor :: Monoid w => (w -> w) -> Writer w a -> Writer w a
censor f writerMonad = pass $ do
    a <- writerMonad
    return (a, f)
```

### MonadWriter type class

The Writer monad and the helper functions presented in the previous two chapters can be also found in the **Control.Monad.Writer** module.

This module is part of the **mtl** package that as said in the previous chapter is extending the **transformers** package.

The module defines the `MonadWriter` type class that contains the functions `writer`, `tell`, `listen` and `pass`.

They are equivalent to the functions describer in the previous chapter.

```haskell
class (Monoid w, Monad m) => MonadWriter w m | m -> w where
    writer :: (a,w) -> m a
    writer ~(a, w) = do
      tell w
      return a

    tell   :: w -> m ()
    tell w = writer ((),w)

    listen :: m a -> m (a, w)

    pass   :: m (a, w -> w) -> m a
    {-# MINIMAL (writer | tell), listen, pass #-}
```

The minimal complete definition are the `listen`, `pass` and either `writer` or `tell` functions. 

Same as for the `MonadReader` type classes you need to use also here the *MultiParamTypeClasses* and *FunctionalDependencies* language pragma.

For our Writer monad we could write an instance of the `MonadWriter` type class as:
```haskell
instance (Monoid w) => MonadWriter w (Writer w) where
    pass :: Monoid w => Writer w (a, w -> w) -> Writer w a
    pass (Writer ((a,f),w)) = Writer (a,f w)

    listen :: Monoid w => Writer w a -> Writer w (a, w)
    listen (Writer (a,w)) = Writer ((a,w),w)
    
    tell :: Monoid w => w -> Writer w ()
    tell msg = Writer ((),msg)
```

Also here you would need to include the language pragmas *FlexibleInstances* and *InstanceSigs*.

What all four language pragmas do was already explained in the Reader monad lesson. 

### Simple example

Let's say you have a friend that has a million EUR and he has no idea about cryptocurrecies. 

You know a crypto currency project where you can stake your money and get 1% interest rate annually. 

The good thing is the cryptocurrency is a fully colaterized stable coin pedged to the value of 1 EUR and does not change over time.

You friend says that he would lend you his money for investing with a 2% annual interest rate that is calculated same way as if you take a credit in a bank.

For a bank credit the customer pays each year a portion of the credit back and the interest rate is applied to the drecreased amout still owed.

You agree and make a deal with him under the condition that he gets his money back including interest rates at the very end of the lending period.

The formula for calculating the final amount your friend gets back including interest rates is:
```
1.000.000 EUR * (1 + 0.02 * (years + 1) / 2)
```

We write now a program that lets us calculate for a number of staking years what will be our profit in the end if we put this money up for staking.

The profit you get is the staking profit minus the interest payments your friend will get because he decided to give you the money for staking. 

The formula for calculating the final amount you get including the staking rewards:
```
1.000.000 EUR * (1 + 0.01) ** years
```

We will write this code with help of two Writer monads, that have a writer variable of type `[String]` and are parameterized by `Int` that represents the final amount.

These monads that calculate the final credit costs and staking amount, depend on the number of staking years, so they will take an `Int` parameter as input.

In [ ]:
import Control.Monad.Writer (runWriter, MonadWriter(tell), Writer)

type Years = Int
type Messages = [String]
type Costs = Int
type Reward = Int

capital = 1000000
interestRate = 0.02
rewardRate = 0.01 

creditCosts :: Years -> Writer Messages Costs
creditCosts years = do
    let costs = (round $ capital * interestRate*(fromIntegral years + 1)/2) :: Int
    tell ["Your credit cost in " ++ show years ++ " years will be: " ++ show costs]
    return costs

stakingRewards :: Years -> Writer Messages Reward
stakingRewards years = do
    let reward = (round $ capital * ((1 + rewardRate) ** fromIntegral years - 1)) :: Int
    tell ["Your final staked amount in " ++ show years ++ " years will be: " ++ show reward]
    return reward

analyzeProfit :: Years -> Writer Messages ()
analyzeProfit years = do  
    cost <- creditCosts years  
    reward <- stakingRewards years
    let profit = reward - cost
    tell ["The profit you make after " ++ show years ++ " years is: " ++ show profit ++ " EUR."]
    return ()

main :: IO ()
main = do
    putStrLn "Input number of years you can afford to take the credit: "
    years <- read <$> getLine :: IO Int
    let (_, messages) = runWriter $ analyzeProfit years
    printList messages

printList :: [String] -> IO ()
printList [] = return ()
printList (x:xs) = do
    putStrLn x 
    printList xs

main

When we use the `tell` function the message gets appended to the writer variable of the `creditCosts` and `stakingRewards` monads. 

When we call them in the `analyzeProfit` Writer monad, its writer variable gets update with the writer variables from those Writer monads. 

The type of the writer parameter which is in our case a list of strings, has to have an instance of the **Monoid** type class, that lists have.

## Writer monad examples

Let's look at a simple example where we use also the `listen` function that enables us to retrieve the logs from the Writer monad.

We input an integer number, pass it on to a Writer monad, do some computation on it and log messages.

In [ ]:
import Control.Monad.Writer (runWriter, MonadWriter(listen, tell), Writer)

type Result = Int
type Logs a = Writer [String] a

logMsg :: String -> Logs ()
logMsg msg = tell [msg]

add1 :: Int -> Logs Result
add1 x = do
    logMsg "Starting add1."
    let y = x + 1
    logMsg $ "Computed result: " ++ show y
    return y

start :: Int -> Logs Result
start x = do
    (n, logs) <- listen $ add1 x
    let logLines = length logs
    logMsg $ "add1 logged " ++ show logLines ++ " lines"
    return n

main :: IO ()
main = do
    print "Input an integer number:"
    n <- (read <$> getLine) :: IO Int
    let (result, logs) = runWriter $ start n
    print $ "Result is: " ++ show result
    putStrLn "Logs are: "
    printList logs

main

This is a simple example but imagine in another example you have to call 10 connected functions where each on average can make up to 3 logs.

And in the last function you call, there is a condition statement that checks weather more then 5 WARNING messages have been logged. 

Let us re-write the example above such that in the `start` function we use the `listens` function instead of `listen`.

In [ ]:
import Control.Monad.Writer (listens, runWriter, MonadWriter(tell), Writer)

start :: Int -> Logs Result
start x = do
    (n, logLines) <- listens length $ add1 x
    logMsg $ "add1 logged " ++ show logLines ++ " lines"
    return n

main

In our next example we demonstrate how the `pass` function works. We write a simple program that adds up to prices and logs messages.

The `pass` function is then used when applying the `transform` function to the logs of the Writer monads. 

In [ ]:
import Control.Monad.Writer (runWriter, MonadWriter(pass, tell), Writer)

logCost :: Double -> Writer [String] Double  
logCost x = do
    tell ["Cost is: " ++ show x ++ " EUR"]
    return x            
      
sumCosts :: Writer [String] (Double, [String] -> [String])
sumCosts = do  
    a <- logCost 7.99 
    b <- logCost 11.99
    tell ["Summed costs are: " ++ show (a + b) ++ " EUR"]
    return (a + b,transform)

process :: Writer [String] Double
process = do
    pass sumCosts

transform :: [String] -> [String]
transform list = map ("NOTE: " ++) list

main :: IO ()
main = do
    let (_, messages) = runWriter process
    printList messages
    
main

Let us re-write the example above such that we use in the `sumCosts` function `censor` instead of the `pass` function. 

Notice how the type signatures of the function `sumCosts` gets simplified. 

In [ ]:
import Control.Monad.Writer (censor, runWriter, MonadWriter(tell), Writer)    
      
sumCosts :: Writer [String] Double
sumCosts = do  
    a <- logCost 7.99 
    b <- logCost 11.99
    tell ["Summed costs are: " ++ show (a + b) ++ " EUR"]
    return $ a + b

process :: Writer [String] Double
process = do
    censor transform sumCosts

main

We can now look at our initial example for the sort functions. Let's re-write it with the use of the Writer monad.

In [ ]:
import Control.Monad.Writer (runWriter, MonadWriter(tell), Writer)

instance Semigroup Int where
  (<>) = (+)

instance Monoid Int where
  mempty = 0
  mappend = (<>)

type List = [Int]

selectionSort :: List -> Writer Int ()
selectionSort []   = return ()
selectionSort list = do
    tell 1
    selectionSort rest
  where rest = filter (/= minimum list) list

quicksort :: List -> Writer Int ()
quicksort []     = return () 
quicksort (x:xs) = do
    tell 1
    quicksort lesser 
    quicksort greater
  where 
    lesser  = filter (< x) xs 
    greater = filter (>= x) xs 

mergeSort :: List -> Writer Int List
mergeSort []  = return []
mergeSort [x] = return [x]
mergeSort xs = do
    tell 1
    xs1 <- mergeSort (firstHalf xs) 
    xs2 <- mergeSort (secondHalf xs)
    return $ merge xs1 xs2
  where merge xs [] = xs
        merge [] ys = ys
        merge (x:xs) (y:ys) | x <= y    = x:merge xs (y:ys)
                            | otherwise = y:merge (x:xs) ys
        firstHalf  xs = let { n = length xs } in take (div n 2) xs
        secondHalf xs = let { n = length xs } in drop (div n 2) xs

testSorts :: [Int] -> IO ()
testSorts list = do
    let (_, count1) = runWriter $ selectionSort list
        (_, count2) = runWriter $ quicksort list
        (_, count3) = runWriter $ mergeSort list 
    
    putStrLn $ "Operation count results for the list " ++ show list
    putStrLn $ "Selection sort: " ++ show count1
    putStrLn $ "Quick sort: " ++ show count2
    putStrLn $ "Merge sort: " ++ show (count3 + 1)
    putStrLn ""

main :: IO ()
main = do
    let list1 = [7, 6, 5, 4, 3, 2, 1]
        list2 = [3, 2, 1, 4, 1, 2, 3]
        list3 = [6, 3, 4, 2, 1, 3, 5]
        
    testSorts list1
    testSorts list2
    testSorts list3

main

First we create a Monoid instance for the `Int` type that allows us to keep track of recursive calls.

The first two sort functions `selectionSort` and `quicksort`, we can implement without passing on the sorted list.

The last `mergeSort` function has to return a sorted list because the algorithm structure demans this when making recursive calls.

Altogether we see that the code for the first two functions becomes more cleaner than in the intial example.

Also a developer that reads our code can easier reason about what it does.

## Recap

In this lesson we've discussed:

- the motivation for introducing the Writer monad 

- the definition of the Writer monad type

- helper functions that work with the Writer monad

- examples that use the Writer monad type and its helper functions